# Model Metrics - Part 1

When a model is generated, it may or may not have 'good' score. Models can score high on training dataset but perform worse on validation dataset. In either case, hyperparameter tuning plays an important role in tweaking the model to avoid overfitting and underfitting. Generalised models perform consistently on any dataset. 

CML Experiments feature provides an mechanism for model tuning with different dataset, features and algorithms. Using Experiments and model metrics, hyperparameter tuning is automated and expedited.

### Hyperparameter Tuning

First step is to create a script that takes range of parameters, trains & scores the model and provide an output of best set of parameters. Metrics generated in an experiment run are tracked by MLFlow and logged in CML. While this can also be achieved using Jupyter notebook, we will create python script for reusability.

In [1]:
import pandas as pd

In [2]:
# Read training dataset
pd_df = pd.read_csv('/home/cdsw/data/UCI_Credit_Card.csv.zip', compression='zip')

In [3]:
cat_cols = ['SEX', 'EDUCATION', 'MARRIAGE', 'AGE_GROUP']
numeric_cols = list(set(pd_df.columns) - set(cat_cols) - set(['ID', 'default.payment.next.month', 'AGE']))

In [4]:
# Data Cleansing
pd_df = pd_df.drop(['ID'], axis=1)

In [5]:
df_clean = pd_df.replace({'EDUCATION': {0 : 5, 6: 5}})
df_clean = df_clean.replace({'MARRIAGE' : {0: 3}})

In [6]:
pay_late_cols = ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5' , 'PAY_6']
for col in pay_late_cols:
    df_clean.loc[df_clean[col] < 0, col] = 0

In [7]:
def get_bill_ratio(df):
    blr_cols = []
    bins= [20, 30, 40, 50, 60, 100]
    labels = [1, 2, 3, 4, 5]
    
    for mth in range(1, 7, 1):
        blr_cols.append(['BLR'+str(mth), 'BILL_AMT'+str(mth)])
    
    for col_blr in blr_cols:
        df[col_blr[0]] = df[col_blr[1]] / df_clean['LIMIT_BAL']

    return df

In [8]:
def get_age_group(df):
    bins= [20, 30, 40, 50, 60, 100]
    labels = [1, 2, 3, 4, 5]
    df['AGE_GROUP'] = pd.cut(df['AGE'], bins=bins, labels=labels, right=False)
    df = df.drop('AGE', axis=1)
    
    return df

In [9]:
# Feature Engineering
df_ftr_1 = get_bill_ratio(df_clean)
df_ftr = get_age_group(df_ftr_1)

In [10]:
#Train Test split
from sklearn.model_selection import train_test_split

X = df_ftr.drop('default.payment.next.month', axis=1).copy()
y = df_ftr['default.payment.next.month'].copy()

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=4)

In [ ]:
# HyperParameter tuning

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

import numpy as np

scaler = Pipeline(
    steps=[("scaler", StandardScaler())]
)

one_hot_encoder = Pipeline(
    steps=[('encoder', OneHotEncoder())]
)

rf_clf = RandomForestClassifier(class_weight = "balanced")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", scaler, numeric_cols),
        ("cat", one_hot_encoder, cat_cols),
    ]
)

clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", rf_clf)]
)

parameters = {'classifier__n_estimators': [100, 150, 200, 250, 300], 'classifier__max_depth': list(range(1, 20, 5)), 
              'classifier__min_samples_split': np.arange(0.1, 1.0, 0.5), 'classifier__min_samples_leaf': np.arange(0.1, 0.5, 0.2)}

optimizer = GridSearchCV(clf, parameters)

model = optimizer.fit(X_train, y_train)

### MLFlow

MLFlow is embedded into CML experiments feature for tracking. Using MLFlow, experiments can be started and metrics & params are tracked for each run. This helps log metrics and compare run results.

Create an experiment from Experiments page in CML project. After creating the experiment, you can then reference the experiment name to track each run of the experiment.

<img src="../docs/Experiments.png" width="625" height="1250">

Below is an example of commands that can be used for tracking metrics and params from script created earlier:

## Experiment runs

Putting all together, [hyperparameter_tuning.py](../scripts/hyperparameter_tuning.py) is created. Run the script and go to Experiments page to monitor the run.

In [15]:
!python ../scripts/3_hyperparameter_tuning.py uci_creditcard_exp0

Successfully registered model 'testmodel'.
2024/03/15 00:56:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: testmodel, version 1
Created version '1' of model 'testmodel'.


For every run, metrics are logged. After running the experiment, navigate to experiment page and look at the runs.

<img src="../docs/Experiment_Runs.png" width="725" height="1250">

Once the runs complete, the metrics and parameters can be compared between runs.

<img src="../docs/Experiment_Compare_Runs.png" width="725" height="1250">

It can be noted that the best performing parameters are logged along with their score. This can then be used for future training.